In [1]:
%pip install xgboost lightgbm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 12.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 11.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.0/199.0 MB 12.3 MB/s eta 0:00:0000:0100:01


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
import xgboost as xgb
import lightgbm as lgb

In [3]:
data = pd.read_csv('CleanedDataForModel.csv')
# data.drop(columns=['Unnamed: 0'],inplace=True)
data

,ITEM_NAME,PRICE_PER_UNIT.$NUMBERDECIMAL,ITEM_CATEGORY,FINAL_RATING,HEALTH_IMPACT_RATING,INGREDIENT_QUALITY_RATING,NUTRITIONAL_CONTENT_RATING,NUTRITION.ENERGY,NUTRITION.PROTEIN,NUTRITION.CARBOHYDRATE,NUTRITION.TOTAL_SUGARS,NUTRITION.ADDED_SUGARS,NUTRITION.TOTAL_FAT,NUTRITION.SATURATED_FAT,NUTRITION.FIBER,NUTRITION.SODIUM
0,"MAGGI 2-MINUTE INSTANT NOODLES - MASALA, 70 G ...",0.200000,INSTANT NOODLES,2.8,2.8,2.5,3.0,384.000,8.2,59.6,1.8,1.3,12.5,8.2,0.0,1028.3
1,KNORR K-POT SPICY KIMCHI VEG MEAL KOREAN RAMEN...,0.625000,INSTANT NOODLES,3.0,2.9,2.8,3.2,384.000,8.2,59.6,1.8,1.3,12.5,8.2,0.0,1028.3
2,INDOMIE INSTANT SOUP NOODLES - SPECIAL CHICKEN...,0.390000,INSTANT NOODLES,3.1,3.0,2.9,3.3,456.000,10.6,64.1,2.6,0.0,18.0,0.0,2.8,0.0
3,SAMYANG FIRE BULDAK HOT CHICKEN FLAVOUR RAMESN...,0.285710,INSTANT NOODLES,3.0,2.8,3.0,3.2,295.000,6.0,42.0,6.0,0.0,10.0,5.0,0.0,630.0
4,NONGSHIM SHIN RAMYUN SPICY FLAVOURED INSTANT N...,0.930233,INSTANT NOODLES,2.8,2.5,2.7,3.1,424.000,8.6,66.0,2.6,0.0,14.0,14.0,0.0,1470.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,HYP MEAL REPLACEMENT WHEY PROTEIN BAR PACK OF ...,2.000000,PROTEIN_BAR,3.8,4.0,3.5,4.0,0.236,20.0,26.7,6.3,0.0,7.8,3.0,5.5,140.0
84,THE WHOLE TRUTH - 12G PROTEIN BAR,1.890000,PROTEIN_BAR,3.7,3.5,4.0,3.5,259.000,12.4,19.1,14.8,0.0,14.9,4.0,2.5,1.4
85,THE WHOLE TRUTH - HAZELNUT COCOA PROTEIN BAR W...,1.562500,PROTEIN_BAR,3.5,3.0,4.0,3.5,265.300,12.1,15.6,11.4,0.0,17.2,27.0,2.2,13.2
86,RITEBITE MAX PROTEIN DAILY CHOCO CLASSIC 10G P...,1.400000,PROTEIN_BAR,3.3,3.5,3.0,3.5,206.000,10.0,22.8,7.9,0.0,7.9,2.9,4.0,139.0


In [4]:
X = data[['NUTRITION.ENERGY', 'NUTRITION.PROTEIN', 'NUTRITION.CARBOHYDRATE', 
          'NUTRITION.TOTAL_SUGARS', 'NUTRITION.ADDED_SUGARS', 'NUTRITION.TOTAL_FAT',
          'NUTRITION.SATURATED_FAT', 'NUTRITION.FIBER', 'NUTRITION.SODIUM']]
y = data['NUTRITIONAL_CONTENT_RATING']


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [6]:
# Hyperparameter Grids for tuning
param_grid_rf = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
}

param_grid_dt = {
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
}

param_grid_xgb = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5, 7]
}

param_grid_lgb = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5, 7]
}


In [7]:
# Model instantiations for GridSearchCV
rf = RandomForestRegressor(random_state=42)
dt = DecisionTreeRegressor(random_state=42)
xgboost = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
lightgbm = lgb.LGBMRegressor(random_state=42)

# Apply GridSearchCV for each model
grid_rf = GridSearchCV(estimator=rf, param_grid=param_grid_rf, cv=3, n_jobs=-1, verbose=2)
grid_dt = GridSearchCV(estimator=dt, param_grid=param_grid_dt, cv=3, n_jobs=-1, verbose=2)
grid_xgb = GridSearchCV(estimator=xgboost, param_grid=param_grid_xgb, cv=3, n_jobs=-1, verbose=2)
grid_lgb = GridSearchCV(estimator=lightgbm, param_grid=param_grid_lgb, cv=3, n_jobs=-1, verbose=2)


In [8]:
# Fit each GridSearchCV model
print("Tuning Random Forest...")
grid_rf.fit(X_train, y_train)
print("Tuning Decision Tree...")
grid_dt.fit(X_train, y_train)
print("Tuning XGBoost...")
grid_xgb.fit(X_train, y_train)
print("Tuning LightGBM...")
grid_lgb.fit(X_train, y_train)

# Best models from GridSearchCV
best_rf = grid_rf.best_estimator_
best_dt = grid_dt.best_estimator_
best_xgb = grid_xgb.best_estimator_
best_lgb = grid_lgb.best_estimator_

Tuning Random Forest...
Fitting 3 folds for each of 12 candidates, totalling 36 fits


[CV] END max_depth=None, min_samples_split=2, n_estimators=100; total time=   0.2s
[CV] END max_depth=None, min_samples_split=5, n_estimators=100; total time=   0.2s
[CV] END max_depth=None, min_samples_split=2, n_estimators=100; total time=   0.2s
[CV] END max_depth=10, min_samples_split=2, n_estimators=100; total time=   0.2s
[CV] END max_depth=None, min_samples_split=2, n_estimators=100; total time=   0.2s
[CV] END max_depth=None, min_samples_split=2, n_estimators=200; total time=   0.4s
[CV] END max_depth=None, min_samples_split=5, n_estimators=100; total time=   0.2s
[CV] END max_depth=None, min_samples_split=5, n_estimators=100; total time=   0.2s
[CV] END max_depth=None, min_samples_split=5, n_estimators=200; total time=   0.4s
[CV] END max_depth=None, min_samples_split=2, n_estimators=200; total time=   0.4s
[CV] END max_depth=10, min_samples_split=2, n_estimators=100; total time=   0.2s
[CV] END max_depth=None, min_samples_split=2, n_estimators=200; total time=   0.4s
[CV] END

In [9]:
voting_regressor = VotingRegressor([
    ('lr', LinearRegression()),  # Using default LinearRegression as is
    ('rf', best_rf),
    ('dt', best_dt),
    ('xgb', best_xgb),
    ('lgb', best_lgb)
], weights=[1, 2, 1, 3, 3])  # Assigning higher weights to stronger models

# Creating a pipeline with StandardScaler and VotingRegressor
pipeline = Pipeline([
    ('scaler', StandardScaler()),    # Standardize the features
    ('voting_regressor', voting_regressor)
])

In [10]:
pipeline.fit(X_train, y_train)

# Predicting the ratings for the test data
y_pred = pipeline.predict(X_test)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000027 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 186
[LightGBM] [Info] Number of data points in the train set: 70, number of used features: 9
[LightGBM] [Info] Start training from score 3.430000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

## With GridSearchCV

In [11]:
y_pred

array([1.88192246, 3.09921178, 4.05012345, 2.76051016, 3.40828585,
       3.96073882, 4.21936474, 2.21234882, 2.6651577 , 3.67276648,
       3.38581811, 2.18073204, 4.21361608, 4.24826445, 3.72894753,
       2.22713052, 4.02172514, 3.08315626])

In [12]:
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse}")

Mean Squared Error (MSE): 0.18153364526910823


## Without GridSearchCV

In [13]:
y_pred

array([1.88192246, 3.09921178, 4.05012345, 2.76051016, 3.40828585,
       3.96073882, 4.21936474, 2.21234882, 2.6651577 , 3.67276648,
       3.38581811, 2.18073204, 4.21361608, 4.24826445, 3.72894753,
       2.22713052, 4.02172514, 3.08315626])

In [14]:
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse}")

Mean Squared Error (MSE): 0.18153364526910823


## Exporting as pickle

In [15]:
import joblib

# Assuming the pipeline has been trained already
# Save the trained model pipeline as a pickle file
joblib.dump(pipeline, 'voting_ensemble_model.pkl')

print("Model exported successfully as voting_ensemble_model.pkl")

Model exported successfully as voting_ensemble_model.pkl
